In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [3]:
cur_folder_name = os.getcwd().split('/')[-1]
if cur_folder_name != "Virny":
    os.chdir("../..")

print('Current location: ', os.getcwd())

Current location:  /Users/denys_herasymuk/UCU/4course_2term/Bachelor_Thesis/Code/Virny


# Multiple Models Interface With Inprocessor

In this example, we are going to audit one inprocessor from AIF360 for stability and fairness, visualize metrics, and create an analysis report. For that, we will use `compute_metrics_with_config` interface that can compute metrics for multiple models. Thus, we will need to do the next steps:

* Initialize input variables

* Compute subgroup metrics

* Make group metrics composition

* Create metrics visualizations and an analysis report

## Import dependencies

In [4]:
import os
from pprint import pprint
from datetime import datetime, timezone

from sklearn.linear_model import LogisticRegression

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

from virny.utils.custom_initializers import create_config_obj, read_model_metric_dfs
from virny.user_interfaces.multiple_models_api import compute_metrics_with_config
from virny.preprocessing.basic_preprocessing import preprocess_dataset
from virny.custom_classes.metrics_visualizer import MetricsVisualizer
from virny.custom_classes.metrics_composer import MetricsComposer

pip install 'aif360[LawSchoolGPA]'


## Initialize Input Variables

Based on the library flow, we need to create 3 input objects for a user interface:

* A **config yaml** that is a file with configuration parameters for different user interfaces for metrics computation.

* A **dataset class** that is a wrapper above the user’s raw dataset that includes its descriptive attributes like a target column, numerical columns, categorical columns, etc. This class must be inherited from the BaseDataset class, which was created for user convenience.

* Finally, a **models config** that is a Python dictionary, where keys are model names and values are initialized models for analysis. This dictionary helps conduct audits for different analysis modes and analyze different types of models.

In [5]:
DATASET_SPLIT_SEED = 42
MODELS_TUNING_SEED = 42
TEST_SET_FRACTION = 0.2

### Create a config object

`compute_metrics_with_config` interface requires that your **yaml file** includes the following parameters:

* **dataset_name**: str, a name of your dataset; it will be used to name files with metrics.

* **bootstrap_fraction**: float, the fraction from a train set in the range [0.0 - 1.0] to fit models in bootstrap (usually more than 0.5).

* **n_estimators**: int, the number of estimators for bootstrap to compute subgroup stability metrics.

* **sensitive_attributes_dct**: dict, a dictionary where keys are sensitive attribute names (including intersectional attributes), and values are disadvantaged values for these attributes. Intersectional attributes must include '&' between sensitive attributes. You do not need to specify disadvantaged values for intersectional groups since they will be derived from disadvantaged values in sensitive_attributes_dct for each separate sensitive attribute in this intersectional pair.

In [6]:
ROOT_DIR = os.path.join('docs', 'examples')
config_yaml_path = os.path.join(ROOT_DIR, 'experiment_config.yaml')
config_yaml_content = """
dataset_name: Law_School
bootstrap_fraction: 0.8
computation_mode: error_analysis
n_estimators: 30  # Better to input the higher number of estimators than 100; this is only for this use case example
sensitive_attributes_dct: {'male': '0.0', 'race': 'Non-White', 'male&race': None}
"""

with open(config_yaml_path, 'w', encoding='utf-8') as f:
    f.write(config_yaml_content)

In [7]:
config = create_config_obj(config_yaml_path=config_yaml_path)
SAVE_RESULTS_DIR_PATH = os.path.join(ROOT_DIR, 'results', f'{config.dataset_name}_Metrics_{datetime.now(timezone.utc).strftime("%Y%m%d__%H%M%S")}')

### Preprocess the dataset and create a BaseFlowDataset class

In [8]:
from virny.datasets.data_loaders import LawSchoolDataset

data_loader = LawSchoolDataset()
data_loader.X_data[data_loader.X_data.columns[:5]].head()

,decile1b,decile3,lsat,ugpa,zfygpa
0,10.0,10.0,44.0,3.5,1.33
1,5.0,4.0,29.0,3.5,-0.11
2,8.0,7.0,37.0,3.4,0.63
3,8.0,7.0,43.0,3.3,0.67
4,3.0,2.0,41.0,3.3,-0.67


In [9]:
column_transformer = ColumnTransformer(transformers=[
    ('categorical_features', OneHotEncoder(handle_unknown='ignore', sparse=False), data_loader.categorical_columns),
    ('numerical_features', StandardScaler(), data_loader.numerical_columns),
])

In [10]:
# Create a binary race column for in-processing since aif360 inprocessors use a sensitive attribute during their learning.
data_loader.X_data['race_binary'] = data_loader.X_data['race'].apply(lambda x: 1 if x == 'White' else 0)

base_flow_dataset = preprocess_dataset(data_loader, column_transformer, TEST_SET_FRACTION, DATASET_SPLIT_SEED)
base_flow_dataset.X_train_val['race_binary'] = data_loader.X_data.loc[base_flow_dataset.X_train_val.index, 'race_binary']
base_flow_dataset.X_test['race_binary'] = data_loader.X_data.loc[base_flow_dataset.X_test.index, 'race_binary']

### Define an inprocessor and create a wrapper for it

To use inprocessors from AIF360 together with Virny, we need to create a wrapper to use it as a basic model in the _models_config_.

A wrapper should include the following methods:
* **fit(self, X, y)**: fits an inprocessor based on X and y pandas dataframes. Returns self.
* **predict_proba(self, X)**: predicts using the fitted inprocessor based on X features pandas dataframe. Returns probabilities for **ZERO** class. These probabilities will be used by Virny in the downstream metric computation.
* **predict(self, X)**: predicts using the fitted inprocessor based on X features pandas dataframe. Returns labels for each sample.
* **__copy__(self)** and **__deepcopy__(self, memo)**: methods, which will be used during copy.copy(inprocessor_wrapper) and copy.deepcopy(inprocessor_wrapper). Return a new instance of inprocessor's wrapper.
* **get_params(self)**: returns parameters of the wrapper. Alignment with sklearn API.

In [24]:
import copy
import numpy as np

from aif360.algorithms.inprocessing import ExponentiatedGradientReduction
from virny.custom_classes.base_inprocessing_wrapper import BaseInprocessingWrapper
from virny.utils.postprocessing_intervention_utils import construct_binary_label_dataset_from_df


class ExpGradientReductionWrapper(BaseInprocessingWrapper):
    """
    A wrapper for fair inprocessors from aif360. The wrapper aligns fit, predict, and predict_proba methods
    to be compatible with sklearn models.

    Parameters
    ----------
    inprocessor
        An initialized inprocessor from aif360.
    sensitive_attr_for_intervention
        A sensitive attribute name to use in the fairness in-processing intervention.

    """

    def __init__(self, inprocessor, sensitive_attr_for_intervention):
        self.sensitive_attr_for_intervention = sensitive_attr_for_intervention
        self.inprocessor = inprocessor

    def fit(self, X, y):
        train_binary_dataset = construct_binary_label_dataset_from_df(X_sample=X,
                                                                      y_sample=y,
                                                                      target_column='target',
                                                                      sensitive_attribute=self.sensitive_attr_for_intervention)
        # Fit an inprocessor
        self.inprocessor.fit(train_binary_dataset)
        return self

    def predict_proba(self, X):
        y_empty = np.zeros(X.shape[0])
        test_binary_dataset = construct_binary_label_dataset_from_df(X_sample=X,
                                                                     y_sample=y_empty,
                                                                     target_column='target',
                                                                     sensitive_attribute=self.sensitive_attr_for_intervention)
        test_dataset_pred = self.inprocessor.predict(test_binary_dataset)
        # Set 1.0 since ExponentiatedGradientReduction can return probabilities slightly higher than 1.0.
        # This can cause Infinity values for entropy.
        test_dataset_pred.scores[test_dataset_pred.scores > 1.0] = 1.0
        # Return 1 - test_dataset_pred.scores since scores are probabilities for label 1, not for label 0
        return 1 - test_dataset_pred.scores

    def predict(self, X):
        y_empty = np.zeros(shape=X.shape[0])
        test_binary_dataset = construct_binary_label_dataset_from_df(X_sample=X,
                                                                     y_sample=y_empty,
                                                                     target_column='target',
                                                                     sensitive_attribute=self.sensitive_attr_for_intervention)
        test_dataset_pred = self.inprocessor.predict(test_binary_dataset)
        return test_dataset_pred.labels

    def __copy__(self):
        new_inprocessor = copy.copy(self.inprocessor)
        return ExpGradientReductionWrapper(inprocessor=new_inprocessor,
                                           sensitive_attr_for_intervention=self.sensitive_attr_for_intervention)

    def __deepcopy__(self, memo):
        new_inprocessor = copy.deepcopy(self.inprocessor)
        return ExpGradientReductionWrapper(inprocessor=new_inprocessor,
                                           sensitive_attr_for_intervention=self.sensitive_attr_for_intervention)

    def get_params(self):
        return {'sensitive_attr_for_intervention': self.sensitive_attr_for_intervention}


In [25]:
# Define a name of a sensitive attribute for the in-processing intervention.
# Note that in the above wrapper, 1 is used as a favorable label, and 0 is used as an unfavorable label.
sensitive_attr_for_intervention = 'race_binary'

# Define an inprocessor
estimator = LogisticRegression(solver='lbfgs', max_iter=1000)
inprocessor = ExponentiatedGradientReduction(estimator=estimator,
                                             constraints='DemographicParity',
                                             drop_prot_attr=True)

models_config = {
    'ExponentiatedGradientReduction': ExpGradientReductionWrapper(inprocessor=inprocessor, 
                                                                  sensitive_attr_for_intervention=sensitive_attr_for_intervention)
}

## Subgroup Metrics Computation

After the variables are input to a user interface, the interface uses subgroup analyzers to compute different sets of metrics for each privileged and disadvantaged subgroup. As for now, our library supports **Subgroup Variance Analyzer** and **Subgroup Error Analyzer**, but it is easily extensible to any other analyzers. When the variance and error analyzers complete metrics computation, their metrics are combined, returned in a matrix format, and stored in a file if defined.

In [13]:
metrics_dct = compute_metrics_with_config(dataset=base_flow_dataset,
                                          config=config,
                                          models_config=models_config,
                                          save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                          notebook_logs_stdout=True)

Analyze multiple models:   0%|          | 0/1 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/30 [00:00<?, ?it/s]

Look at several columns in top rows of computed metrics

In [14]:
sample_model_metrics_df = metrics_dct[list(models_config.keys())[0]]
sample_model_metrics_df[sample_model_metrics_df.columns[:6]].head(20)

,Metric,overall,male_priv,male_priv_correct,male_priv_incorrect,male_dis
0,Std,0.010754,0.009248,0.005442,0.048157,0.012742
1,IQR,0.011812,0.010275,0.006811,0.045689,0.013844
2,Mean_Prediction,0.024874,0.022293,0.014474,0.102233,0.028284
3,Overall_Uncertainty,0.024440,0.020417,0.012212,0.104294,0.029756
4,Aleatoric_Uncertainty,0.009367,0.007112,0.004255,0.036311,0.012347
5,Statistical_Bias,0.098595,0.089850,0.003912,0.968374,0.110150
6,Epistemic_Uncertainty,0.015073,0.013305,0.007957,0.067982,0.017409
7,Jitter,0.009843,0.008588,0.005536,0.039789,0.011501
8,Label_Stability,0.986587,0.988232,0.992273,0.946919,0.984412
9,TPR,0.989807,0.989767,1.000000,0.000000,0.989861


## Group Metrics Composition

**Metrics Composer** is responsible for this second stage of the model audit. Currently, it computes our custom group fairness and stability metrics, but extending it for new group metrics is very simple. We noticed that more and more group metrics have appeared during the last decade, but most of them are based on the same subgroup metrics. Hence, such a separation of subgroup and group metrics computation allows one to experiment with different combinations of subgroup metrics and avoid subgroup metrics recomputation for a new set of grouped metrics.

In [15]:
models_metrics_dct = read_model_metric_dfs(SAVE_RESULTS_DIR_PATH, model_names=list(models_config.keys()))

In [16]:
metrics_composer = MetricsComposer(models_metrics_dct, config.sensitive_attributes_dct)

Compute composed metrics

In [17]:
models_composed_metrics_df = metrics_composer.compose_metrics()

In [18]:
models_composed_metrics_df

,Metric,male,race,male&race,Model_Name
0,Accuracy_Parity,-0.021944,-0.179603,-0.159554,ExponentiatedGradientReduction
1,Aleatoric_Uncertainty_Parity,0.005235,0.034924,0.041620,ExponentiatedGradientReduction
2,Aleatoric_Uncertainty_Ratio,1.736181,9.635121,7.862084,ExponentiatedGradientReduction
3,Epistemic_Uncertainty_Parity,0.004104,0.013899,0.019680,ExponentiatedGradientReduction
4,Epistemic_Uncertainty_Ratio,1.308413,2.072889,2.456499,ExponentiatedGradientReduction
5,Equalized_Odds_FNR,-0.000093,0.000639,0.002249,ExponentiatedGradientReduction
6,Equalized_Odds_FPR,-0.011832,0.082345,0.057266,ExponentiatedGradientReduction
7,IQR_Parity,0.003570,0.021962,0.026651,ExponentiatedGradientReduction
8,Jitter_Parity,0.002913,0.014650,0.017267,ExponentiatedGradientReduction
9,Label_Stability_Ratio,0.996135,0.980762,0.977664,ExponentiatedGradientReduction


## Metrics Visualization and Reporting

**Metric Visualizer** allows us to build static visualizations for the computed metrics. It unifies different preprocessing methods for the computed metrics and creates various data formats required for visualizations. Hence, users can simply call methods of the _MetricsVisualizer_ class and get custom plots for diverse metric analysis.

In [19]:
visualizer = MetricsVisualizer(models_metrics_dct, models_composed_metrics_df, config.dataset_name,
                               model_names=list(models_config.keys()),
                               sensitive_attributes_dct=config.sensitive_attributes_dct)

In [20]:
visualizer.create_overall_metrics_bar_char(
    metric_names=['Accuracy', 'F1', 'TPR', 'TNR', 'PPV', 'Selection-Rate'],
    plot_title="Accuracy Metrics"
)

alt.Chart(...)

In [21]:
visualizer.create_overall_metrics_bar_char(
    metric_names=['Aleatoric_Uncertainty', 'Overall_Uncertainty', 'Label_Stability', 'Std', 'IQR', 'Jitter'],
    plot_title="Stability and Uncertainty Metrics"
)

alt.Chart(...)